
# K-IFRS RAG — HF v2 (Cached-only QA)

**요청대로: 벡터 재생성 없이, 이미 저장된 임베딩 `.npy`만 사용**해 QA를 수행하는 버전입니다.  
- 타이틀/문단 임베딩은 `/mnt/data/hf_cache/*.npy`에서 **로드만** 합니다.  
- 캐시가 없으면 **오류를 던지고 중단**합니다(실수로 재생성되지 않도록).  
- 쿼리 임베딩만 모델로 생성(필수)하며, 문서 임베딩은 절대 재계산하지 않습니다.

내장 기능: **BM25 + 벡터 하이브리드**, **Two-Track 라우팅**, **Vector-only Fallback**, **Alias 확장**  
(재랭커는 선택: 필요시 주석 해제)


In [1]:

from pathlib import Path
import json, math, numpy as np
from collections import defaultdict, Counter
from typing import List

# ===== 경로 =====
BASE = Path('/Users/igangsan/Desktop/기준서')  # 필요시 수정
JSON_PATH = BASE / 'kifrs_combined_2_cleaned_v2_final4.json'
CACHE_DIR = BASE / 'hf_cache_3'
TITLE_EMB_PATH = CACHE_DIR / 'title_emb_intfloat_multilingual-e5-large.npy'
PARA_EMB_PATH  = CACHE_DIR / 'para_emb_intfloat_multilingual-e5-large.npy'

print('JSON exists:', JSON_PATH.exists(), JSON_PATH)
print('Cache dir exists:', CACHE_DIR.exists(), CACHE_DIR)
print('Title emb exists:', TITLE_EMB_PATH.exists())
print('Para emb exists :', PARA_EMB_PATH.exists())

# 캐시가 없으면 바로 에러 (재생성 금지)
if not (TITLE_EMB_PATH.exists() and PARA_EMB_PATH.exists()):
    raise FileNotFoundError('임베딩 캐시(.npy)가 없습니다. 재생성을 허용하지 않는 cached-only 모드입니다. 캐시를 먼저 준비해 주세요.')


JSON exists: True /Users/igangsan/Desktop/기준서/kifrs_combined_2_cleaned_v2_final4.json
Cache dir exists: True /Users/igangsan/Desktop/기준서/hf_cache_3
Title emb exists: True
Para emb exists : True


In [2]:

# ===== JSON 로드 & 인덱스 구성 =====
with JSON_PATH.open(encoding='utf-8') as f:
    data = json.load(f)

docs = data.get('documents', [])
total_paras = sum(len(d.get('paragraphs', [])) for d in docs)
print('Loaded standards:', len(docs), '| Total paragraphs:', total_paras)

title_keys = []
title_texts = []
title_to_para_indices = defaultdict(list)
paragraphs = []

for d in docs:
    std = d.get('standard_no')
    ttl = d.get('title') or ''
    src = d.get('source_file') or ''
    key = (std, ttl, src)
    if key not in title_keys:
        title_keys.append(key)
        head = (d.get('paragraphs', [{}])[:3])
        head_txt = " ".join([(p.get('text') or '') for p in head])
        title_texts.append(f"{ttl}\n{src}\n{head_txt[:1000]}")
    base_idx = len(paragraphs)
    for p in d.get('paragraphs', []):
        paragraphs.append({
            "std": std, "title": ttl, "source": src,
            "page": p.get('page'), "para_id": p.get('para_id'), "text": p.get('text') or ''
        })
        title_to_para_indices[key].append(base_idx); base_idx += 1

print('Titles:', len(title_keys), '| Paragraphs:', len(paragraphs))


Loaded standards: 28 | Total paragraphs: 2145
Titles: 28 | Paragraphs: 2145


In [3]:

# ===== 토크나이저 & BM25 =====
import re
HAN_ENG_NUM = re.compile(r'[가-힣A-Za-z0-9]+', re.UNICODE)
STOP = set(['그리고','등','및','또는','그러나','이는','그','이','저','것','수','등의'])

def normalize(t: str) -> str:
    return t.strip().lower()

def tokenize(t: str):
    return HAN_ENG_NUM.findall(normalize(t))

def filter_tokens(tokens: List[str]) -> List[str]:
    return [w for w in tokens if w not in STOP and len(w) > 1]

class BM25:
    def __init__(self, docs_tokens, k1=1.5, b=0.75):
        self.docs_tokens = docs_tokens
        self.N = len(docs_tokens)
        self.k1 = k1; self.b = b
        self.avgdl = sum(len(d) for d in docs_tokens) / max(1, self.N)
        self.df = Counter()
        for doc in docs_tokens:
            for term in set(doc):
                self.df[term] += 1
        self.idf = {t: math.log(1 + (self.N - df + 0.5)/(df + 0.5)) for t, df in self.df.items()}
        self.tf = [Counter(doc) for doc in docs_tokens]

    def _score_doc(self, q_tokens, i):
        score, tf, dl = 0.0, self.tf[i], len(self.docs_tokens[i])
        for term in q_tokens:
            idf = self.idf.get(term)
            if idf is None: 
                continue
            f = tf.get(term, 0)
            if f == 0: 
                continue
            denom = f + self.k1 * (1 - self.b + self.b * dl / self.avgdl)
            score += idf * (f * (self.k1 + 1)) / denom
        return score

    def search(self, q_tokens, topk=50):
        scores = []
        for i in range(self.N):
            s = self._score_doc(q_tokens, i)
            if s != 0.0:
                scores.append((i, s))
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:topk]

    def search_subset(self, q_tokens, allowed: set, topk=50):
        scores = []
        for i in allowed:
            s = self._score_doc(q_tokens, i)
            if s != 0.0:
                scores.append((i, s))
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:topk]

# BM25 인덱스 (토큰만 필요)
title_tokens = [filter_tokens(tokenize(t)) for t in title_texts]
bm25_title = BM25(title_tokens)

para_texts = [p['text'] for p in paragraphs]
para_tokens = [filter_tokens(tokenize(t)) for t in para_texts]
bm25_para = BM25(para_tokens)


In [4]:

# ===== 캐시 임베딩 로드 (문서 임베딩만) =====
title_vecs = np.load(TITLE_EMB_PATH)
para_vecs  = np.load(PARA_EMB_PATH)
print('Loaded embeddings:', title_vecs.shape, para_vecs.shape)


Loaded embeddings: (28, 1024) (2145, 1024)


In [5]:

# ===== 쿼리 임베딩 모델 (문서 임베딩 재생성 없음) =====
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = "intfloat/multilingual-e5-large"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_queries(texts: List[str]) -> np.ndarray:
    inputs = [f"query: {t}" for t in texts]
    return embed_model.encode(inputs, normalize_embeddings=True, convert_to_numpy=True, batch_size=64)


/opt/anaconda3/envs/fintech/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

# ===== 유틸 & Alias =====
def topk_by_cosine(q_vec: np.ndarray, mat: np.ndarray, k: int = 50):
    sims = (mat @ q_vec)  # 정규화 가정
    idxs = np.argpartition(-sims, kth=min(k, len(sims)-1))[:k]
    idxs = idxs[np.argsort(-sims[idxs])]
    return [(int(i), float(sims[i])) for i in idxs]

ALIAS = {
    "개발비": ["개발활동", "무형자산"],
    "감가상각": ["정액법", "체감잔액법", "생산량비례법"],
    "매출": ["수익"],
    "선수수익": ["계약부채"],
    "미수수익": ["계약자산"],
    "비연결재무제표": ["별도재무제표"],
}

def expand_query(q: str) -> str:
    terms = set()
    for k, syns in ALIAS.items():
        if k in q:
            terms.update(syns)
    if terms:
        return q + " " + " ".join(sorted(terms))
    return q


In [7]:

# ===== 라우팅 & 검색 (Two-Track + Vector-only Fallback) =====
def route_titles(query: str, topn: int = 5, w_bm25=0.4, w_vec=0.6, debug=False):
    q_exp = expand_query(query)
    qtok = filter_tokens(tokenize(q_exp))
    bm_hits = bm25_title.search(qtok, topk=max(topn*6, 30))

    qv = embed_queries([q_exp])[0]
    vec_hits = topk_by_cosine(qv, title_vecs, k=max(topn*6, 30))
    vec_dict = dict(vec_hits)

    combined = []
    seen = set([i for i,_ in bm_hits])
    for i, s_bm in bm_hits:
        s_vec = vec_dict.get(i, 0.0)
        combined.append((i, w_bm25*s_bm + w_vec*s_vec, s_bm, s_vec))
    for i, s_vec in vec_hits:
        if i in seen: 
            continue
        combined.append((i, w_vec*s_vec, 0.0, s_vec))

    combined.sort(key=lambda x: x[1], reverse=True)
    routed = combined[:topn]
    if debug:
        print("[Routing] top candidates:")
        for r in routed:
            i, s, sb, sv = r
            std, ttl, src = title_keys[i]
            print(f" - {std} | {ttl} | score={s:.3f} (bm25={sb:.3f}, vec={sv:.3f})")
    return routed

def hybrid_search_within_titles(query: str, title_idxs: List[int],
                                topk_bm25=150, topk_final=12,
                                w_bm25=0.4, w_vec=0.6,
                                use_reranker: bool = False, rerank_top: int = 20,
                                debug=False):
    allowed = set()
    for ti in title_idxs:
        allowed.update(title_to_para_indices[title_keys[ti]])
    if debug:
        print(f"[Within] allowed paragraphs: {len(allowed)}")

    if not allowed:
        if debug: print("[Within] allowed empty -> fallback to global search")
        return global_hybrid_search(query, topk_bm25=200, topk_final=15,
                                    w_bm25=w_bm25, w_vec=w_vec,
                                    use_reranker=use_reranker, rerank_top=rerank_top)

    q_exp = expand_query(query)
    qtok = filter_tokens(tokenize(q_exp))
    bm_hits = bm25_para.search_subset(qtok, allowed, topk=topk_bm25)

    # ★ BM25=0이면 벡터만으로 랭킹
    if not bm_hits:
        if debug: print("[Within] BM25 hits=0 -> vector-only ranking")
        qv = embed_queries([q_exp])[0]
        idxs = np.array(list(allowed), dtype=int)
        sims = para_vecs[idxs] @ qv                    # sims.shape == (len(idxs),)
        order = np.argsort(-sims)[:topk_final]         # order: sims 내부 인덱스
        chosen = idxs[order]                           # 전역 문단 인덱스로 매핑

        results = [
    {**paragraphs[int(j)], "score": float(sims[int(i)]), "bm25": 0.0, "vector": float(sims[int(i)])}
    for i, j in zip(order, chosen)
                    ]
        return results

    qv = embed_queries([q_exp])[0]
    vec_scores = {i: float(np.dot(para_vecs[i], qv)) for i, _ in bm_hits}

    combined = []
    for i, s_bm in bm_hits:
        s_vec = vec_scores.get(i, 0.0)
        combined.append((i, w_bm25*s_bm + w_vec*s_vec, s_bm, s_vec))
    combined.sort(key=lambda x: x[1], reverse=True)
    combined = combined[:topk_final]

    results = [{**paragraphs[i], "score": s, "bm25": s_bm, "vector": s_vec} for (i, s, s_bm, s_vec) in combined]
    return results

def global_hybrid_search(query: str, topk_bm25=200, topk_final=15,
                         w_bm25=0.4, w_vec=0.6, use_reranker=False, rerank_top=20, debug=False):
    q_exp = expand_query(query)
    qtok = filter_tokens(tokenize(q_exp))
    bm_hits = bm25_para.search(qtok, topk=topk_bm25)

    if not bm_hits:
        if debug: print("[Global] BM25 hits=0 -> vector-only ranking")
        qv = embed_queries([q_exp])[0]
        sims = np.dot(para_vecs, qv)
        order = np.argsort(-sims)[:topk_final]
        return [{**paragraphs[i], "score": float(sims[i]), "bm25": 0.0, "vector": float(sims[i])} for i in order]

    qv = embed_queries([q_exp])[0]
    vec_scores = {i: float(np.dot(para_vecs[i], qv)) for i, _ in bm_hits}

    combined = []
    for i, s_bm in bm_hits:
        s_vec = vec_scores.get(i, 0.0)
        combined.append((i, w_bm25*s_bm + w_vec*s_vec, s_bm, s_vec))
    combined.sort(key=lambda x: x[1], reverse=True)
    combined = combined[:topk_final]
    return [{**paragraphs[i], "score": s, "bm25": s_bm, "vector": s_vec} for (i, s, s_bm, s_vec) in combined]

def hierarchical_search_two_track(query: str, top_titles=4, **kw):
    print(f"[Query] {query}")
    routed = route_titles(query, topn=top_titles, debug=True)
    title_idx = [i for (i, *_ ) in routed]
    if not title_idx:
        print("[Two-Track] routed empty -> GLOBAL")
        return routed, global_hybrid_search(query, **kw)
    results = hybrid_search_within_titles(query, title_idx, **kw)
    return routed, results

def print_titles(routed):
    for rank, (i, s, s_bm, s_vec) in enumerate(routed, 1):
        std, ttl, src = title_keys[i]
        print(f"{rank:>2}. [{std}] {ttl}  <{src}>  score={s:.3f} (bm25={s_bm:.3f}, vec={s_vec:.3f})")


In [8]:

# ===== Demo (캐시 사용 QA) =====
queries = [
    #"유형자산의 감가상각의 방법에는 무엇이 있나요?",
    "개발비 자산 인식 요건은 무엇인가요?"
]

for q in queries:
    print("\n=== Q:", q)
    routed, results = hierarchical_search_two_track(q, top_titles=4,
                                                    topk_bm25=150,
                                                    topk_final=10,
                                                    w_bm25=0.4, w_vec=0.6,
                                                    use_reranker=False)
    print("Top titles:")
    print_titles(routed)
    for i, r in enumerate(results[:5], 1):
        snippet = r['text'].replace('\n', ' ')
        if len(snippet) > 240: snippet = snippet[:240] + '…'
        print(f"  {i:>2}. [{r['std']}:{r['para_id']}] ({r['title']}) p.{r.get('page','?')} score={r['score']:.3f}")
        print("     ", snippet)



=== Q: 개발비 자산 인식 요건은 무엇인가요?
[Query] 개발비 자산 인식 요건은 무엇인가요?
[Routing] top candidates:
 - 1103 | 사업결합 | score=2.724 (bm25=5.570, vec=0.827)
 - 1038 | 무형자산 | score=2.013 (bm25=3.751, vec=0.854)
 - 1036 | 자산손상 | score=1.526 (bm25=2.577, vec=0.825)
 - 1023 | 차입원가 | score=1.245 (bm25=1.862, vec=0.834)
Top titles:
 1. [1103] 사업결합  <K-IFRS_제1103호_사업결합.pdf>  score=2.724 (bm25=5.570, vec=0.827)
 2. [1038] 무형자산  <K-IFRS_제1038호_무형자산.pdf>  score=2.013 (bm25=3.751, vec=0.854)
 3. [1036] 자산손상  <K-IFRS_제1036호_자산손상.pdf>  score=1.526 (bm25=2.577, vec=0.825)
 4. [1023] 차입원가  <K-IFRS_제1023호_차입원가.pdf>  score=1.245 (bm25=1.862, vec=0.834)
   1. [1038:5] (무형자산) p.3 score=5.284
      이 기준서는 광고, 교육훈련, 사업개시, 연구와 개발활동 등에 대한지출에 적용한다. 연구와 개발활동의 목적은 지식의 개발에 있다.따라서 이러한 활동으로 인하여 물리적 형체(예: 시제품)가 있는 자산이 만들어지더라도, 그 자산의 물리적 요소는 무형자산 요소 즉,그 자산이 갖는 지식에 부수적인 것으로 본다.
   2. [1038:57] (무형자산) p.18 score=4.566
      다음 사항을 모두 제시할 수 있는 경우에만 개발활동(또는 내부 프로젝트의 개발단계)에서 발생한 무형자산을 인식한다.⑴ 무형자산을 사용하거나 판매하

In [9]:
# ====== Imports ======
import os
from openai import OpenAI

# ====== 환경변수 세팅 ======
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"   # ← 맨 위에 추가
client = OpenAI()  # 이후부터는 client 사용 가능

In [10]:
# ===== OpenAI LLM 연결 (환경변수 OPENAI_API_KEY 필요) =====
from openai import OpenAI
import os, re

OPENAI_MODEL = os.environ.get("OPENAI_MODEL", "gpt-4o-mini")
openai_client = OpenAI()

def _join_chunks(chunks, max_chars=1800):
    buf = []
    total = 0
    for ch in chunks:
        tag = f"[{ch.get('std')}:{ch.get('para_id')}] {ch.get('title')} p.{ch.get('page','?')}"
        txt = ch.get('text','').strip().replace("\n"," ")
        entry = tag + "\n" + txt
        if total + len(entry) > max_chars:
            break
        buf.append(entry)
        total += len(entry)
    return "\n\n".join(buf)


In [11]:
# ===== LLM 재랭킹 + 답변 생성 =====
def llm_rerank(query: str, candidates: list, top_m: int = 6):
    """LLM을 사용해 상위 후보들 중 정답 근거 가능성이 높은 문단을 재선정.
    '가끔 1위가 오답, 하위가 정답' 문제를 완화합니다."""
    # 후보들을 메타+본문으로 정리
    packed = []
    for c in candidates:
        meta = f"[{c.get('std')}:{c.get('para_id')}] ({c.get('title')}) p.{c.get('page','?')} score={c.get('score',0):.3f}"
        text = c.get('text','').strip().replace("\n", " ")
        packed.append({"meta": meta, "text": text})
    # 번호와 함께 구성
    numbered = []
    for i, it in enumerate(packed, 1):
        numbered.append(f"{i}. {it['meta']}\n{it['text']}")
    blocks = "\n\n".join(numbered)

    prompt = f"""
당신은 엄격한 근거 기반 심사관입니다.
질문과 후보 문단 목록을 보고, 질문에 가장 정확한 '근거'가 되는 문단 순서 Top-{top_m}만 고르세요.
반드시 번호만 콤마로 출력하세요. (예: 7,3,1,5,2)

[질문]
{query}

[후보 문단]
{blocks}
"""

    chat = openai_client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "너는 사실검증 심사관이다. 허상 없이 근거가 있는 후보만 고른다."},
            {"role": "user",   "content": prompt}
        ],
        temperature=0
    )
    content = chat.choices[0].message.content.strip()
    # 번호 파싱
    picked = []
    for tok in re.findall(r"\d+", content):
        i = int(tok)
        if 1 <= i <= len(candidates):
            picked.append(i-1)
    # 중복 제거, 상위 top_m 유지
    order=[]; seen=set()
    for i in picked:
        if i not in seen:
            order.append(i); seen.add(i)
    order = order[:top_m] if len(order) >= top_m else order + [i for i in range(len(candidates)) if i not in seen][:top_m-len(order)]
    return [candidates[i] for i in order]

def answer_with_rag_llm(query: str, *, top_titles=4, topk_bm25=150, topk_final=15, rerank_take=6, w_bm25=0.4, w_vec=0.6, debug=False):
    # 1) 1차 검색 (하향식 라우팅 + 문단 하이브리드)
    routed, results = hierarchical_search_two_track(query, top_titles=top_titles,
                                                    topk_bm25=topk_bm25,
                                                    topk_final=topk_final,
                                                    w_bm25=w_bm25, w_vec=w_vec,
                                                    use_reranker=False)
    if debug:
        print("[RAG] first-stage results:", len(results))
    # 2) LLM 재랭킹 (상위 후보 중 '정답 근거'를 재선정)
    chosen = llm_rerank(query, results[:max(10, rerank_take*2)], top_m=rerank_take)
    if debug:
        print("[Rerank] chosen:", [f"{c.get('std')}:{c.get('para_id')}" for c in chosen])
    # 3) 컨텍스트 구성
    ctx = _join_chunks(chosen, max_chars=2200)
    qa_prompt = f"""
당신은 K-IFRS 회계 기준 전문가입니다.
아래 [근거 문단]만을 토대로 질문에 간결하고 정확하게 답하세요.
근거에 숫자/연도가 있으면 한국식 표기 유지.
필요하면 핵심 근거를 한두 문장 인용하되, 문맥을 바꾸지 마세요.

[질문]
{query}

[근거 문단]
{ctx}
"""

    chat = openai_client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role":"system","content":"너는 한국어로 답변하는 회계 기준 전문가다. 허상 금지, 모르면 모른다고 말한다."},
            {"role":"user","content": qa_prompt}
        ],
        temperature=0.2
    )
    answer = chat.choices[0].message.content.strip()
    return {"answer": answer, "evidence": chosen, "titles": routed}


In [12]:
# ===== Demo 2: LLM 기반 재랭크 + 답변 =====
demo_queries = [
    "자본변동표에서 2016년 총계는?",
    "개발비 자산 인식 요건은 무엇인가요?"
]
for q in demo_queries:
    print("\n=== Q:", q)
    out = answer_with_rag_llm(q, debug=True)
    print("\n[Answer]\n", out["answer"])
    print("\n[Evidence]")
    for e in out["evidence"]:
        print(f"- [{e['std']}:{e['para_id']}] {e['title']} p.{e.get('page','?')}")



=== Q: 자본변동표에서 2016년 총계는?
[Query] 자본변동표에서 2016년 총계는?
[Routing] top candidates:
 - 1016 | 유형자산 | score=0.468 (bm25=0.000, vec=0.780)
 - 1021 | 환율변동효과 | score=0.467 (bm25=0.000, vec=0.779)
 - 1008 | 회계정책, 회계추정치 변경과 오류 | score=0.465 (bm25=0.000, vec=0.774)
 - 1007 | 현금흐름표 | score=0.464 (bm25=0.000, vec=0.774)
[RAG] first-stage results: 1
[Rerank] chosen: ['1007:60']

[Answer]
 근거 문단에는 2016년 총계에 대한 정보가 포함되어 있지 않습니다. 따라서 2016년 총계에 대한 답변을 드릴 수 없습니다.

[Evidence]
- [1007:60] 현금흐름표 p.20

=== Q: 개발비 자산 인식 요건은 무엇인가요?
[Query] 개발비 자산 인식 요건은 무엇인가요?
[Routing] top candidates:
 - 1103 | 사업결합 | score=2.724 (bm25=5.570, vec=0.827)
 - 1038 | 무형자산 | score=2.013 (bm25=3.751, vec=0.854)
 - 1036 | 자산손상 | score=1.526 (bm25=2.577, vec=0.825)
 - 1023 | 차입원가 | score=1.245 (bm25=1.862, vec=0.834)
[RAG] first-stage results: 15
[Rerank] chosen: ['1038:57', '1038:5', '1038:31', '1038:124', '1038:3', '1038:74']

[Answer]
 개발비 자산 인식 요건은 다음과 같습니다:

1. 무형자산을 사용하거나 판매하기 위해 그 자산을 완성할 수 있는 기술적 실현가능성.
2. 무형자산을 완성하여 사용하거나 판매